In [ ]:
import pandas as pd
import numpy as np

### `Datetime` 인덱스
시계열 객체인 `Series`는 시계열 데이터를 관리하는 객체이기 때문에 데이터가 시간에 의해 관리되어져야함.  

지금까지는 단순한 인덱스로 사용하였는데 이를 시계열로 표현하기 위해서 `DatetimeIndex`를 사용해야함.  

`DatetimeIndex`는 시계열 인덱스를 사용하기 위한 인덱스이며,  
`pandas.to_datetime`, `pandas.date_range` 메서드를 사용하여 생성할 수 있음.

`to_datetime()` 메서드는 날짜 및 시간을 나타내는 문자열을 Datetime 자료형으로 변경한 후 `DatetimeIndex`를 생성함

In [ ]:
date_str = ['2018, 1, 1', '2018, 1, 4', '2018, 1, 5', '2018, 1, 6']
idx = pd.to_datetime(date_str)
idx

In [ ]:
# 인덱스 타입 : datetime 타입
s = pd.Series(np.random.randn(4),index=idx)
s

In [ ]:
# 인덱스 타입 : string 타입
s = pd.Series(np.random.randn(4),index=date_str)
s

`date_range` 메서드는 날짜 및 시간에 대해서 시작일과 종료일 또는 시작일과 기간을 입력하여  
범위 내의 `DatetimeIndex` 를 생성

In [ ]:
pd.date_range('2024-01-01', '2024-03-31')

In [ ]:
pd.date_range('2024-01-01',periods=30)

`freq` 인수로 특정한 날짜만 생성되도록 할 수 있음

In [ ]:
# s : 초
pd.date_range('2024-02-29 09:48:00', '2024-02-29 09:48:10', freq='s')

In [ ]:
# B : 평일
pd.date_range('2024-02-01','2024-02-29',freq='B')

In [ ]:
# QE-DEC : 각 분기별 마지막 날
pd.date_range('2024-01-01','2024-12-31',freq='QE-DEC')

### `shift` 연산
`shift()` 메서드는 인덱스는 그대로 두고 데이터만 이동  
`freq` 인수를 지정하면 인덱스를 변경시킴

In [ ]:
ts = pd.Series(np.random.randn(4),index=pd.date_range('2024-02-01',periods=4, freq='ME'))
ts

In [ ]:
ts.shift(1) # VALUE열 1칸 아래로 내리기

In [ ]:
ts.shift(-1) # VALUE열 1칸 위로 올리기

In [ ]:
ts.shift(2,freq='ME') # INDEX열 두칸 밀기

### `resample` 메서드
`resample()` 메서드는 `DatetimeIndex`의 시간 간격을 재조정함.  

시간 간격이 좁은 단위로 변경하면 데이터 양이 증가해서 up-sampling이라고 하고
넓은 단위로 변경하면 데이터 양이 감소해서 down-sampling이라고 함.

In [ ]:
ts = pd.Series(np.random.randn(100),index=pd.date_range('2024-01-01',periods=100))
ts.tail(20)

down-sampling 시에는 기존 데이터가 그룹화되는 경우와 같기 때문에 대표값을 지정해야함 (집계처리를 해야함)

In [ ]:
ts.resample('W').mean()

시간(시/분) 단위에서는 가장 빠른 값은 포함하고 가장 늦은 값은 포함하지 않음

In [ ]:
ts = pd.Series(np.arange(60),index=pd.date_range('2024-02-29',periods=60,freq='min'))
ts

In [ ]:
ts.resample('10min').min()

In [ ]:
ts.resample('10min').max()

구한 한계값을 가장 늦은 값으로 지정하고자 한다면 `closed='right'`를 사용함

In [ ]:
ts.resample('10min',closed='right').min()

`ohlc()` 메서드는 구간의 시점, 고점, 저점, 종점의 값을 표현

In [21]:
ts.resample('10min').ohlc()

,open,high,low,close
2024-02-29 00:00:00,0,9,0,9
2024-02-29 00:10:00,10,19,10,19
2024-02-29 00:20:00,20,29,20,29
2024-02-29 00:30:00,30,39,30,39
2024-02-29 00:40:00,40,49,40,49
2024-02-29 00:50:00,50,59,50,59


In [22]:
ts.resample('10min',closed='right').ohlc()

,open,high,low,close
2024-02-28 23:50:00,0,0,0,0
2024-02-29 00:00:00,1,10,1,10
2024-02-29 00:10:00,11,20,11,20
2024-02-29 00:20:00,21,30,21,30
2024-02-29 00:30:00,31,40,31,40
2024-02-29 00:40:00,41,50,41,50
2024-02-29 00:50:00,51,59,51,59


`up-sampling`은 데이터를 추가해야하기 때문에 forward filling 방식과 back filling 방식을 사용할 수 있음.  

forward filling은 이전 데이터를 가져와 사용하는 방법 - `ffill()`  
back filling은 다음 데이터를 가져와 사용하는 방법 - `bfill()`

In [24]:
ts.resample('30s').ffill()

2024-02-29 00:00:00     0
2024-02-29 00:00:30     0
2024-02-29 00:01:00     1
2024-02-29 00:01:30     1
2024-02-29 00:02:00     2
                       ..
2024-02-29 00:57:00    57
2024-02-29 00:57:30    57
2024-02-29 00:58:00    58
2024-02-29 00:58:30    58
2024-02-29 00:59:00    59
Freq: 30s, Length: 119, dtype: int32

In [25]:
ts.resample('30s').bfill()

2024-02-29 00:00:00     0
2024-02-29 00:00:30     1
2024-02-29 00:01:00     1
2024-02-29 00:01:30     2
2024-02-29 00:02:00     2
                       ..
2024-02-29 00:57:00    57
2024-02-29 00:57:30    58
2024-02-29 00:58:00    58
2024-02-29 00:58:30    59
2024-02-29 00:59:00    59
Freq: 30s, Length: 119, dtype: int32

### `dt` 접근자
datetime의 데이터 타입 시리즈는 `dt` 접근자를 사용하여 datetime 데이터 타입이 가지는 여러가지 속성 및 메서드를 사용할 수 있음

In [26]:
s = pd.Series(pd.date_range('2024-01-01',periods=10))
s

0   2024-01-01
1   2024-01-02
2   2024-01-03
3   2024-01-04
4   2024-01-05
5   2024-01-06
6   2024-01-07
7   2024-01-08
8   2024-01-09
9   2024-01-10
dtype: datetime64[ns]

`year`, `month`, `day` 와 같은 속성을 사용할 수 있음

In [28]:
s.dt.day

0     1
1     2
2     3
3     4
4     5
5     6
6     7
7     8
8     9
9    10
dtype: int32

In [29]:
s.dt.year

0    2024
1    2024
2    2024
3    2024
4    2024
5    2024
6    2024
7    2024
8    2024
9    2024
dtype: int32

`strftime()` 메서드를 사용하여 datetime 데이터 타입의 데이터를 문자열 데이터로 변경할 수 있음.

In [32]:
s.dt.strftime('%Y.%M.%d. %H:%M%s')

0    2024-01-01 00:00:00
1    2024-01-02 00:00:00
2    2024-01-03 00:00:00
3    2024-01-04 00:00:00
4    2024-01-05 00:00:00
5    2024-01-06 00:00:00
6    2024-01-07 00:00:00
7    2024-01-08 00:00:00
8    2024-01-09 00:00:00
9    2024-01-10 00:00:00
dtype: object

In [58]:
df = pd.DataFrame({
    "date": pd.date_range("2020-12-25", periods=100, freq="D"),
    "value": np.random.randint(100, size=(100,))
})
df

,date,value
0,2020-12-25,68
1,2020-12-26,57
2,2020-12-27,43
3,2020-12-28,31
4,2020-12-29,23
...,...,...
95,2021-03-30,70
96,2021-03-31,37
97,2021-04-01,24
98,2021-04-02,52


In [59]:
df.date = df.date.dt.month
df

,date,value
0,12,68
1,12,57
2,12,43
3,12,31
4,12,23
...,...,...
95,3,70
96,3,37
97,4,24
98,4,52


In [60]:
df.groupby(df.date).sum()

,value
date,
1,1566
2,1228
3,1601
4,113
12,320
